In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit
import ar1_approx as ar
import time
from scipy.optimize import root as root
from math import sqrt
%matplotlib inline

# Bonus Problem

### Parameters

In [13]:
# Household parameters
beta = 0.96
h = 6.616
r = (1/beta)-1
# Firm parameters
alpha_k = 0.297
alpha_l = 0.65
delta = 0.154
psi = 1.08
betafirm = 1/(1+r)
# Parameters regarding shock
sigma_eps = 0.213
mu = 0.0
rho = 0.7605
sizez = 9

### Capital Stock Grid

In [15]:
# Discretizing the space of capital stock 
dens = 1
# put in bounds here for the capital stock space
kstar = ((((1 / betafirm - 1 + delta) * ((1 / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (1** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 3*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
@jit
def create_K(numb,dens,delta,ub_k):
    for j in range(int(numb * dens)):
        K[j] = ub_k * (1 - delta) ** (j / dens)
    return K

K = create_K(numb,dens,delta,ub_k)
kvec = K[::-1]
sizek = kvec.shape[0]

### Productivity Shocks

In [16]:
# Find the Markov process for the shocks
z_grid, pi = ar.addacooper(sizez,mu,rho,sigma_eps)
z_grid = np.exp(z_grid) #make it to ln

In [7]:
def find_wage(w):
    